# Cloud Computing: Capacity Control Optimization (Timothy Manolias)

### The following program determines how Cirrus, a cloud computing provider, should allocate its computational capacity to users in order to maximize revenue.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url='Images/Question.png', width=700)

## Solution

### Part 1: Capacity Control Formulation

In [2]:
Image(url='Images/Linear_Program_Formulation.png', width=500)

### Part 2: Solving the Capacity Control Problem

In [4]:
'''Implements Capacity Control Formulation Listed Above Using Gurobi Optimizer.'''

from gurobipy import *

cpu = [16, 32, 64, 8, 16, 32, 16, 32, 64]
memory = [8, 16, 32, 32, 64, 128, 16, 32, 64]
gpu = [1, 1, 1, 1, 1, 1, 2, 6, 8]
price = [7, 12, 24, 22, 44, 88, 30, 90, 120]
rates = [5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3]
T = 5

# Creates the model
m = Model()

# Suppresses output
m.Params.outputflag = 0

# Adds the decision variables
x = m.addVars(9)

# Creates the constraints
cpu_constr    = m.addConstr(sum(x[i]*cpu[i] for i in range(9)) <= 512)
memory_constr = m.addConstr(sum(x[i]*memory[i] for i in range(9)) <= 1024)
gpu_constr    = m.addConstr(sum(x[i]*gpu[i] for i in range(9)) <= 64)
forecast_constr = m.addConstrs((x[i] <= T*rates[i] for i in range(9)))
non_negativity_constr = m.addConstrs((x[i] >= 0 for i in range(9)))

# Creates the objective function [maximize revenue]
m.setObjective(sum(x[i]*price[i] for i in range(9)), GRB.MAXIMIZE)

m.update()
m.optimize()

In [5]:
'''Extracts Solution Status and Optimal Objective Value.'''

# Solution status
status = m.status
if status == GRB.OPTIMAL:
    print('Solved to Optimality!')

# Objective value
optimal_obj = m.objval
print(f'\nOptimal Objective Value: ${optimal_obj:,.2f}')

Solved to Optimality!

Optimal Objective Value: $1,039.43


### Part 3: Simulating Cirrus' Current Suboptimal Policy

In [6]:
Image(url='Images/Current Policy.png', width=700)

In [7]:
'''Generates Random User Arrival Sequence for Cloud Computing Service.'''

import numpy as np
import random

def generateArrivalSequences(nSimulations, rates, T):
    
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = []

    for s in range(nSimulations):
        single_arrival_sequence_time = []
        single_arrival_sequence_type = []
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p=[rate/total_rate for rate in rates])

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    
    return arrival_sequences_times, arrival_sequences_types

**Average Revenue for Current Policy Over 100 Simulated Sequences:**

In [8]:
'''Implements Cirrus' "First-Come First-Serve" Policy.'''

np.random.seed(10)

nSimulations = 100
nResources = 3
B = np.array([512, 1024, 64])

arrival_sequences_times, arrival_sequences_types = generateArrivalSequences(nSimulations, rates, T)

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nSimulations, nResources))

for s in range(nSimulations):
    b = B.copy();
    
    single_revenue = 0.0; # Contains the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s])

    # Iterates through arrivals in sequence
    for j in range(nArrivals):
        if 0 in b:
            break
        
        # Obtains the time of the arrival and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Checks if there is sufficient capacity for the request
        request = [cpu[i], memory[i], gpu[i]]
        if (request[0] <= b[0] and request[1] <= b[1] and request[2] <= b[2]):
            # Accrues revenue and removes the capacity
            single_revenue += price[i]
            for x in range(3):
                b[x] -= request[x]            

    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[s] = b

# Average revenue
mean_myopic_revenue = results_myopic_revenue.mean()
print(f"Cirrus' Average Revenue for Current Policy: ${mean_myopic_revenue:,.2f}")

Cirrus' Average Revenue for Current Policy: $528.28


### Part 4: Bid-Price Control Policy

In [9]:
'''Implements Bid-Price Control (BPC) Policy.'''

def bpc(b, t, k):
    constraints = [cpu_constr, memory_constr, gpu_constr]
    for r in range(nResources):
        constraints[r].RHS = b[r]

    for i in range(9):
        forecast_constr[i].RHS = (T-t) * rates[i]

    # Re-solves the model
    m.update()
    m.optimize()

    # Obtains dual values and shadow prices
    dual_val = [cpu_constr.pi*cpu[k], memory_constr.pi*memory[k], gpu_constr.pi*gpu[k]]

    return dual_val

In [10]:
'''Simulates Average Revenue for BPC Policy.'''

results_revenue = np.zeros(nSimulations)
results_remaining_capacity = np.zeros((nSimulations, nResources))

for s in range(nSimulations):
    
    b = B.copy() # Initializes the current capacity
    single_revenue = 0.0 # Initialize the revenue garnered in this simulation
    nArrivals = len(arrival_sequences_times[s])
    
    for j in range(nArrivals):
        
        # Takes the next arrival time and type from the sequence
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]
        
        # Checks if there is enough capacity
        request = [cpu[i], memory[i], gpu[i]]
        if (request[0] <= b[0] and request[1] <= b[1] and request[2] < b[2]):
            # Re-solves the LO and obtains the dual values
            dual_val = bpc(b, arrival_time, i)

            # Checks if the revenue is at least the sum of the bid prices:
            if sum(dual_val) <= price[i]:
                # Accrues revenue and removes the capacity
                single_revenue += price[i]
                for x in range(3):
                    b[x] -= request[x]

    results_revenue[s] = single_revenue
    results_remaining_capacity[s] = b

mean_revenue = results_revenue.mean()
print(f"Average Revenue for Bid-Price Control Policy: ${mean_revenue:,.2f}")

Average Revenue for Bid-Price Control Policy: $925.59
